In [12]:
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
parent_path = 'C:\\Users\\SIU856533724\\OneDrive - Southern Illinois University\Research\\Paper\\Social Networks\\Trend Prediction\\Comment-Section-UI\\UI\\angular-interactive-comments-section\\flask\\data'

In [13]:
import os
file = os.path.join(parent_path, 'fox-unknown.csv')
df = pd.read_csv(file)

In [14]:
df.head(20)

,Unnamed: 0,comment,time,username,id,timestamp,time_diff
0,31,Glad to hear the UK has some common sense,"19 Dec, 2023 08:09 PM",justjoshin’,32,2023-12-19 20:09:00,0 days 00:00:00
1,109,Thank God Fox keeps us up to date on culture w...,"19 Dec, 2023 08:11 PM",usmckurt1,110,2023-12-19 20:11:00,0 days 00:02:00
2,89,Why would anyone want to be a teacher? It's ki...,"19 Dec, 2023 08:13 PM",Gaza.Glass.workx.LLC,90,2023-12-19 20:13:00,0 days 00:04:00
3,18,If a student has a gender request it means the...,"19 Dec, 2023 08:13 PM",betty_davis_eyes,19,2023-12-19 20:13:00,0 days 00:04:00
4,15,"Nobody, regardless of position relative to the...","19 Dec, 2023 08:15 PM",exmark,16,2023-12-19 20:15:00,0 days 00:06:00
5,7,"""the complex phenomenon of the increasing numb...","19 Dec, 2023 08:16 PM",drachenfels99,8,2023-12-19 20:16:00,0 days 00:07:00
6,90,Home0s want to teach,"19 Dec, 2023 08:16 PM",betty_davis_eyes,91,2023-12-19 20:16:00,0 days 00:07:00
7,25,The only thing a teacher should ever say to an...,"19 Dec, 2023 08:16 PM",perogi007,26,2023-12-19 20:16:00,0 days 00:07:00
8,9,Finally some common sense from the British. Ho...,"19 Dec, 2023 08:16 PM",patriotforever937328,10,2023-12-19 20:16:00,0 days 00:07:00
9,47,"If they aren't over 18yrs, there should NEVER ...","19 Dec, 2023 08:17 PM",usmc18317,48,2023-12-19 20:17:00,0 days 00:08:00


In [15]:
list(df.iloc[1][1:])

['Thank God Fox keeps us up to date on culture war news in the UK! ',
 '19 Dec, 2023 08:11 PM',
 'usmckurt1',
 110,
 '2023-12-19 20:11:00',
 '0 days 00:02:00']

In [16]:
def get_so_far_comments(ith_comment):
    # row_index = df.loc[df['id'] == comment_id].index[0]
    ids = df.iloc[0:ith_comment+1]['id']
    return list(ids)


In [17]:
# get_so_far_comments(1)

In [18]:
df_trend = pd.read_csv(os.path.join(parent_path, 'fox-unknown-trends.csv'))

In [19]:
def get_single_trend_score(trend):
    print("label: ", trend)
    return list(df_trend[trend])

In [20]:
# get_single_trend_score('toxic')

In [21]:
from score_aggregation import get_emotion_trends_polarity

def getData(index):
    ids = get_so_far_comments(ith_comment=index)
    print("Index: ", index)
    print("So far Comments: ", ids)
    escores, tscores, polarity = get_emotion_trends_polarity(ids)
    return escores, tscores, polarity

In [22]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import json

app = Flask(__name__)
cors = CORS(app, resources={r"/api/*": {"origins": "*"}})

@app.route('/api/new-comment', methods=['POST'])
def welcome():
    comment = request.json['content']
    img = request.json['img']
    isYou = request.json['isYou']
    score = request.json['score']
    username = request.json['username']
    print("new: ", comment)
    return 'Successfully comment added!'

@app.route('/api/data/<int:id>', methods=['GET'])
def getApiData(id: int):
    escores, tscores, polarity = getData(id)
    return json.dumps({"escores": escores, "tscores": tscores, "polarity": polarity})

@app.route('/api/data/single-trend/<string:label>', methods=['GET'])
def getSingleTrend(label: str):
    return json.dumps({"singlerend": get_single_trend_score(label)})

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Dec/2023 00:01:22] "GET /api/data/0 HTTP/1.1" 200 -


Index:  0
So far Comments:  [32]


127.0.0.1 - - [22/Dec/2023 00:01:29] "GET /api/data/single-trend/toxic HTTP/1.1" 200 -


label:  toxic


...

label:  obscene


127.0.0.1 - - [22/Dec/2023 14:08:24] "GET /api/data/single-trend/toxic HTTP/1.1" 200 -


label:  toxic


127.0.0.1 - - [22/Dec/2023 14:08:25] "GET /api/data/single-trend/approval HTTP/1.1" 200 -


label:  approval
